<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2

import warnings
warnings.filterwarnings('ignore')

In [2]:
''' Необходимые для подключения к БД данные содержатся в отдельном файле
    "creds.py" для безопасности. Параметры импортируются в словарь CONN_ATTR,
    который далее используется в функции "make_request()" при запросах к базе.
'''
import creds as c_

CONN_ATTR = {
    'user': c_.USER,
    'password': c_.PASSWORD,
    'host': c_.HOST,
    'port': c_.PORT,
    'dbname': c_.DBNAME
    }

def make_request(query_: str) -> object:
    ''' Делает запросы к БД с определёнными аттрибутами
    Arguments:
        query_ [str] -- Строка SQL-запроса (можно многострочник)
    Returns:
        [object] -- Ответ на запрос
    '''
    with psycopg2.connect(**CONN_ATTR) as conn:
        return pd.read_sql_query(query_, conn)


# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [3]:
# текст запроса
query_3_1 = f'''
SELECT COUNT(*)
FROM public.vacancies
'''

In [4]:
# результат запроса
display(make_request(query_3_1))

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [5]:
# текст запроса
query_3_2 = f'''
SELECT COUNT(*)
FROM public.employers
'''

In [6]:
# результат запроса
display(make_request(query_3_2))

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [7]:
# текст запроса
query_3_3 = f'''
SELECT COUNT(*)
FROM public.areas
'''

In [8]:
# результат запроса
display(make_request(query_3_3))

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [9]:
# текст запроса
query_3_4 = f'''
SELECT COUNT(*)
FROM public.industries
'''

In [10]:
# результат запроса
display(make_request(query_3_4))

,count
0,294


***

#### Выводы по предварительному анализу данных ####

Пока ничего особенного. Примерно две вакансии на работодателя ещё ни о чём не говорит. Будем проводить дополнительные исследования ниже.

Интересно посмотреть на распределение вакансий по отраслям и по географии.


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [19]:
# текст запроса
query_4_1 = f'''
SELECT
    a.name AS area,
    COUNT(*) AS cnt
FROM public.vacancies AS v
JOIN public.areas AS a ON
    v.area_id = a.id
GROUP BY a.id
ORDER BY cnt DESC
LIMIT 5
'''

In [20]:
# результат запроса
display(make_request(query_4_1))

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [13]:
# текст запроса
query_4_2 = f'''
SELECT
    COUNT(*)
FROM public.vacancies
WHERE
    salary_from IS NOT NULL
    OR
    salary_to IS NOT NULL
'''

In [14]:
# результат запроса
display(make_request(query_4_2))

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [17]:
# текст запроса
query_4_3 = f'''
SELECT
    -- Применяется преобразование к целочисленному типу.
    -- Для надёжности предварительно используется round() для округления,
    -- хотя можно и без него.
    ROUND(AVG(salary_from))::integer AS avg_min,
    ROUND(AVG(salary_to))::integer AS avg_max
FROM public.vacancies
'''

In [18]:
# результат запроса
display(make_request(query_4_3))

,avg_min,avg_max
0,71065,110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [48]:
# текст запроса
query_4_4 = f'''
SELECT
    schedule,
    employment,
    COUNT(*) AS cnt
FROM public.vacancies
GROUP BY schedule, employment
ORDER BY cnt DESC
LIMIT 5
'''

In [49]:
# результат запроса
display(make_request(query_4_4))

,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [50]:
# текст запроса
query_4_5 = f'''
SELECT
    experience AS "Требуемый опыт работы"
FROM public.vacancies
GROUP BY experience
ORDER BY COUNT(*)
'''

In [51]:
# результат запроса
display(make_request(query_4_5))

,Требуемый опыт работы
0,Более 6 лет
1,Нет опыта
2,От 3 до 6 лет
3,От 1 года до 3 лет


***

#### Выводы по детальному анализу вакансий ####

В лидирующую пятёрку лидеров-регионов ожидаемо вошли столичные города РФ, РБ и РК. Интересно то, что &laquo;столица&raquo; Сибири (Новосибирск) обогнала &laquo;столицу&raquo; Урала (Екатеринбург). Это может означать, что в СФО началось опережающее развитие, либо просто на Урале больше крупных городов, по которым &laquo;распыляются&raquo; вакансии федерального округа. Жаль, что для уточнения в БД нет дополнительных региональных идентификаторов.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [53]:
# текст запроса
query_5_1 = f'''
SELECT
    e.name,
    COUNT(*) AS vacancies_number
FROM public.vacancies AS v
JOIN public.employers AS e ON
    v.employer_id = e.id
GROUP BY e.name
ORDER BY vacancies_number DESC
LIMIT 5
'''

In [54]:
# результат запроса
display(make_request(query_5_1))

,name,vacancies_number
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [ ]:
# текст запроса

In [ ]:
# результат запроса

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [ ]:
# текст запроса

In [ ]:
# результат запроса

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [ ]:
# текст запроса

In [ ]:
# результат запроса

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [ ]:
# текст запроса

In [ ]:
# результат запроса

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [ ]:
# текст запроса

In [ ]:
# результат запроса

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [ ]:
# код для получения списка городов-милионников

In [ ]:
# текст запроса

In [ ]:
# результат запроса

***

In [ ]:
# выводы по анализу работодателей

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [ ]:
# текст запроса

In [ ]:
# результат запроса

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [ ]:
# текст запроса

In [ ]:
# результат запроса

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
# текст запроса

In [ ]:
# результат запроса

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
# текст запроса

In [ ]:
# результат запроса

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
# текст запроса

In [ ]:
# результат запроса

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [ ]:
# текст запроса

In [ ]:
# результат запроса

***

In [ ]:
# выводы по предметному анализу

# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования